## Table of Contents
1. Import Libraries and Load Data
2. Data Filtering
3. Data Cleaning

## 1. Import Libraries and Load Data

In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import skimpy
from summarytools import dfSummary

In [5]:
df_co2 = pd.read_csv('/Users/samabrams/Data Analysis Projects/CO2_Emissions_Project/02 Data/Original Data/visualizing_global_co2_data.csv')

In [7]:
df_co2.shape

(50598, 79)

In [9]:
# Dataframe Summary with dfSummary
co2_summary = dfSummary(df_co2)
co2_summary

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,country[object],1. Lower-middle-income countries2. Tuvalu3. Brunei4. Cambodia5. South America6. Cote d'Ivoire7. Iceland8. Singapore9. Sierra Leone10. North America11. other,"272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)47,878 (94.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADfElEQVR4nO3cwWoaUQCG0WuQ0GxGRPBhfIg8bB/CtxFxFk3Ixi5qd60JNP2k9pxVoZtZfIzTy/27OJ/PAwoPt34A/h9iIyM2MmIjIzYyYiMjNjJiIyM2MosxxtMYY5zP55cbPwt3brnZbJ7HGGOxWHwVHH/TwzRN62ma1mOMx1s/DPfNNxsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkVnO83y8/Pntpk/C3bNBILPwX2ZR8WYjY11FxrqKjKMPMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kXAsnY/BCxpuNjMELGYMXMo4+yIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLyQcS2cjMELGd9sZPyMkrGuImNdRcY3GxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGesqMq6Fk7GuIuPNRsbghYzBCxlHH2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyLgWTsbghYw3GxmDFzIGL2QcfZARGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGesqMjYIZKyryHizkbGuImNdRcbRBxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGYMXMq6FkzF4IePNRsbghYzBCxlHH2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyLgWTsbghYxvNjJ+RslYV5GxriLjm42M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYx1FRnXwslYV5HxZiNj8ELG4IWMow8yYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2RcCydj8ELm55vNlXA+w9u1X8jlZrN5vmwQ4I/M83y8NpxaTtO03u12L6vV6rV+OO7H6XT6st/v14fD4XGM8evYxhhjtVq9brfbb+nTcY+erv2lczYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyyzF+XA+59YPwb/tIQ8t5no/7/X493rkeAu+5jKd+O5xyLZzPdPVauMELGf9AICM2MmIjIzYyYiMjNjJiIyM2MmIj8x0gQBAnkyXa7AAAAABJRU5ErkJggg=="">",0(0.0%)
2,year[int64],Mean (sd) : 1925.4 (59.9)min < med < max:1750.0 < 1929.0 < 2021.0IQR (CV) : 93.0 (32.2),272 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACnklEQVR4nO3du24aQRiG4X9JDINBu+IQuICUdEjbcAEuc7G5BBcUrinSpKJDQogVG6NwSpFYUSQbMzsQ+Fbv06LfrDSvloOYcXQ4HAy4dZVrXwBwio/XvgCURxRFdTOrFhz/eTgcnt96kFBxFlEU1Tudzpc4jltF5rMsW0RR9PWtWAm1ZALvahUz2xecjZvNZm80GmVJkqx9BpfLpRuPx635fF41M0L1ccUFD5l17Xb7IUmShu/gbre7y/P8c7PZ/F6pVDa+85vNxu12u0GtVnvs9Xo/fOfNrH7sQUJ9RcjLWMiCnyMWMxukafrU7XYzn9npdNqaTCafhsPht36/v/B97j/z99vt9oPv7CkI9XXVOI5bo9Ho2fdlLGTBzxWLc27je1dbLBZ1M7NGo7Euckd8mb8UQj0iSRLvRQtZ8FuP5Zr4HhUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSSns+auDR5vF+v7875/UgTClDDf0PHS/n0a/X60czK3IePc6slKFawNHmZpc/jx7+yhqqmRU72tys3EeMq+LDFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiTc9O9RA7aTsJWkZC4aauC+Jddutx+SJGn4DrKVpHzeDTUgtsKhmf2OzcwGaZo+dbvdzGeWrSTlczTUkE1yIaGZ/Y3NObfx3U7CVpLyee+OWniTXEhoZsSGf530HrXIJjlCwznx9RQkECokECokECoknPRharlcOt8/vFqtnJlZnuduNpvd/8951edWve7Q+VP6+gWkZT2kqh3LFAAAAABJRU5ErkJggg=="">",0(0.0%)
3,iso_code[object],1. nan2. AUS3. BRN4. PNG5. KHM6. TUV7. GUF8. GLP9. GBR10. GIN11. other,"8,456 (16.7%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)272 (0.5%)39,694 (78.4%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAD0UlEQVR4nO3cQWrbWgCG0esiQjORMAYvxovIYt8itBsTrEETMvEbNF2CPqfNOQuIBPmQHXH/HO73+4DCj0ffAN+H2MiIjYzYyIiNjNjIiI2M2MiIjcxhjPE8xngKrvVxv9/fguvwRU2n0+llnufj3hfatu31cDj8J7jva5rn+Xi5XN6WZXnf6yK32+3nuq7H6/X6NMYQ2zc1jTHGsizv5/P5187Xet755/PF+QOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjITGP8Pm+250X2/vn8HaZt217XdT2Onc+bbdv2Osb42PMafG02CGQO/mUWlT9PtuGpw96m0+n0MsYYlk/s7cc8z8fPKV/xvY1vzHs2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy0+cQZQxjFHbmWDgZgxcynmxkDF7IGLyQ8eqDjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2MwQsZx8LJGLyQ8Z2NjI9RM

In [15]:
pd.set_option("display.max_rows", None)
df_co2['year'].value_counts().sort_index()

year
1750     34
1751     34
1752     34
1753     34
1754     34
1755     34
1756     34
1757     34
1758     34
1759     34
1760     34
1761     34
1762     34
1763     34
1764     34
1765     34
1766     34
1767     34
1768     34
1769     34
1770     34
1771     34
1772     34
1773     34
1774     34
1775     34
1776     34
1777     34
1778     34
1779     34
1780     34
1781     34
1782     34
1783     34
1784     34
1785     35
1786     35
1787     35
1788     35
1789     35
1790     35
1791     35
1792     37
1793     37
1794     37
1795     37
1796     37
1797     37
1798     37
1799     37
1800     39
1801     39
1802     41
1803     39
1804     40
1805     39
1806     39
1807     40
1808     39
1809     39
1810     40
1811     40
1812     40
1813     40
1814     40
1815     40
1816     40
1817     40
1818     40
1819     41
1820     41
1821     41
1822     41
1823     41
1824     41
1825     41
1826     41
1827     41
1828     41
1829     42
1830     60
1831     60
1832     59

## 2. Filtering Data

In [20]:
df_co2_time = df_co2.copy()[(df_co2['year'] >=1851) & (df_co2['year'] <= 2019)]

In [22]:
df_co2_time.shape

(45900, 79)

In [24]:
df_co2.shape

(50598, 79)

In [40]:
# Create new dataframe filtered to 1851-2019 - 1851 is the first year with complete data for most countries, 2019 is the last pre-pandemic year
# Also filtering for several key columns instead of pulling all 49
columns_to_keep = ['country', 'year', 'iso_code', 'co2', 'co2_per_capita', 'co2_growth_abs', 'coal_co2', 'oil_co2', 'gas_co2', 'cumulative_co2', 'population']
df_co2_time = df_co2.copy().loc[(df_co2['year'] >= 1851) & (df_co2['year'] <= 2019), columns_to_keep]

In [42]:
df_co2_time.shape

(45900, 11)

In [44]:
# Summary of new dataframe
co2_time_summary = dfSummary(df_co2_time)
co2_time_summary

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,country[object],1. Afghanistan2. Niue3. North Macedonia4. North Korea5. North America (excl. USA)6. North America (GCP)7. North America8. Non-OECD (GCP)9. Nigeria10. Qatar11. other,"169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)44,210 (96.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADfUlEQVR4nO3cMWobUQBF0S8jTNyMEAItRovwYrMI7UYITREbN0oRpwhRAgH7DijnVCHVFJfR5PNfVtfrdUDhYekH4P8hNjJiIyM2MmIjIzYyYiMjNjJiI7MaYzyNMcb1en1Z+Fm4c+vdbvc8xhir1eqr4PhMD9M0badp2o4xHpd+GO6bbzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiI7Oe5/n8/ue3RZ+Eu2eDQGblv8yi4s1GxrqKjHUVGUcfZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDF7IuBZOxuCFjDcbGYMXMgYvZBx9kBEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXgh41o4GYMXMr7ZyPgZJWNdRca6ioxvNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZFwLJ2NdRcabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxlvNjIGL2QMXsg4+iAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZGwQyFhXkfFmI2NdRca6ioyjDzJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MgYvZFwLJ2PwQsabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxnfbGT8jJKxriJjXUXGNxsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxnrKjKuhZOxriLjzUbG4IWMwQsZRx9kxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGQMXsi4Fk7G4IXMzzebK+F8pLdbv5Tr3W73/L5BgA8xz/P51oBqPU3T9nA4vGw2m9elHo77cblcvhyPx+3pdHocY/wa2xhjbDab1/1+/22Rp+MePd36S+dsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZNZj/LgWsvSDcB/+1tJ6nufz8Xjcjj9cC4F/9T6i+m1A5Vo4n+HmtXCDFzL+gUBGbGTERkZsZMRGRmxkxEZGbGTERuY7CLkQJ3942gEAAAAASUVORK5CYII="">",0(0.0%)
2,year[int64],Mean (sd) : 1935.1 (48.8)min < med < max:1851.0 < 1935.0 < 2019.0IQR (CV) : 84.0 (39.7),169 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACjUlEQVR4nO3ZP2/aUBSH4XMhoMuf2iKhYWPpGKkDUhaWbh37Ybt26xCkzhm6dMoSRUIGhBuEY+jSpVEKXC4J+SnvM3O45viV5ShuvV4b8NpVjn0BwC5Ojn0B+JdzrmFm9YivWK7X6/tDXc9rsTXUyMVVzGy152zsfOzZe9/wiJ3509PTz2matvY518xsMpnMnXPfzGyxx/gx79fGfW8M1TnXODs7+5IkSSf01LIsa3mef2i3278qlUrxkvOxZ5uZzWazzDn3NTTWmJ0VReHN7OLy8vJHt9udhc7f3t6+G41Gn/r9frdarQaFesz7ZbZ939ueqPUkSTrD4fA+TdOgH35zc9O5vr5+PxgMfvZ6vSxkNnY+9uzpdOqvrq464/G4bmahT9XYnTW998X5+fnvwHMty7KG9745GAwWL72zmPld9r3TO2qapovQxWVZ1jAza7VawbOx87Fn/9XYc87M4nYW6xg7O8DON/52/pj6j7Isa2aWOOdCR5PValV7hkt60wj1CXme14qi+Njv96uh73pFUfiyLC8Wi8V3M9v3aY5HCPUJy+XyJPJdr/nw8FB9rut7iwh1g5h3PRwW/5mCBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFBEKFhJNdPjSdTn3oF8/nc29mlue5v7u7a77kvOrZqtcdO79LX38ANVpT3i8MKjQAAAAASUVORK5CYII="">",0(0.0%)
3,iso_code[object],1. nan2. PAK3. NLD4. ANT5. NCL6. NZL7. NIC8. NER9. NGA10. NIU11. other,"6,692 (14.6%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)169 (0.4%)37,687 (82.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADzUlEQVR4nO3cQU7bUABF0Z/KQmViK4qUxWQRLLaL8G4iFA8KYpIOSneAL0WcswAHiSvHWP9xuN/vAwo/PvsH4PsQGxmxkREbGbGRERsZsZERGxmxkTmMMR7HGA87Xf/tfr+/7HRtvpjpdDo9zfN83OPi27Y9Hw6HX4JjjDGmeZ6Pl8vlZVmW14+88O12+7mu6/F6vT6MMcTGmMYYY1mW1/P5/HuH6z/ucE2+KH8gkBEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbmWmMv2fPPvrCe1yTr23atu15Xdfj2OHs2bZtz2OMt4++Ll+TDQKZg3+ZReXfnW24A7G36XQ6PY0xhhUUe/sxz/Pxfcq313MbjDG8ZyMkNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIzO9j1LGMExhZ46FkzF4IePORsbghYzBCxmvPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEsnIzBCxnPbGR8jZKxriJjXUXGMxsZs

In [32]:
df_co2_time['country'].value_counts()

country
Afghanistan                                      169
Niue                                             169
North Macedonia                                  169
North Korea                                      169
North America (excl. USA)                        169
North America (GCP)                              169
North America                                    169
Non-OECD (GCP)                                   169
Nigeria                                          169
Qatar                                            169
Niger                                            169
Nicaragua                                        169
New Zealand                                      169
New Caledonia                                    169
Netherlands Antilles                             169
Netherlands                                      169
Norway                                           169
OECD (GCP)                                       169
OECD (Jones et al. 2023)              

In [46]:
# Remove countries that don't have a full set of entries for all years
places_to_remove = [
    "Panama Canal Zone (GCP)",
    "St. Kitts-Nevis-Anguilla (GCP)",
    "Ryukyu Islands (GCP)",
    "French Equatorial Africa (GCP)",
    "French West Africa (GCP)",
    "Leeward Islands (GCP)",
    "Kuwaiti Oil Fires (GCP)"]

df_co2_time2 = df_co2_time[~df_co2_time['country'].isin(places_to_remove)]


In [50]:
df_co2_time2.shape

(45799, 11)

In [52]:
# Check for Missing Values
df_co2_time2.isnull().sum().sort_values(ascending=True)

country               0
year                  0
iso_code           6591
population         8338
co2               19036
cumulative_co2    20895
co2_growth_abs    21156
co2_per_capita    21544
oil_co2           22756
coal_co2          22828
gas_co2           22853
dtype: int64

In [58]:
df_co2_time2.to_csv('/Users/samabrams/Data Analysis Projects/CO2_Emissions_Project/02 Data/Prepared Data/co2 time series.csv')

### Filtering for Most Complete Data

Because there are so many missing values, I'm going to further filter this data down to only those countries with the most complete datasets. I'm

In [63]:
# Group by country and count non-null values for `co2` and total years.
country_data = df_co2_time2.groupby('country').agg(
    co2_count=('co2', 'count'),
    total_years=('year', 'nunique'))

In [65]:
# Identify countries with complete data for 'co2'
complete_data_countries = country_data[country_data['co2_count'] == country_data['total_years']]

In [67]:
# Print the countries with complete data
if complete_data_countries.empty:
    print("No countries have complete data for the 'co2' column.")
else:
    print("Countries with complete data for the 'co2' column:")
    print(complete_data_countries.index.to_list())

Countries with complete data for the 'co2' column:
['Africa', 'Africa (GCP)', 'Andorra', 'Asia', 'Asia (GCP)', 'Asia (excl. China and India)', 'Australia', 'Austria', 'Belgium', 'Brunei', 'Cambodia', 'Canada', 'Central America (GCP)', "Cote d'Ivoire", 'Denmark', 'Europe', 'Europe (GCP)', 'Europe (excl. EU-27)', 'Europe (excl. EU-28)', 'European Union (27)', 'European Union (27) (GCP)', 'European Union (28)', 'France', 'French Guiana', 'Germany', 'Guadeloupe', 'Guinea', 'High-income countries', 'Hungary', 'Iceland', 'Low-income countries', 'Lower-middle-income countries', 'Martinique', 'Mauritania', 'Mayotte', 'Middle East (GCP)', 'Netherlands', 'Non-OECD (GCP)', 'North America', 'North America (GCP)', 'North America (excl. USA)', 'Norway', 'OECD (GCP)', 'Oceania', 'Oceania (GCP)', 'Papua New Guinea', 'Poland', 'Reunion', 'Sierra Leone', 'Singapore', 'South America', 'South America (GCP)', 'Spain', 'Sweden', 'Tuvalu', 'United Kingdom', 'United States', 'Upper-middle-income countries', '

In [69]:
# Count Countries
len(complete_data_countries)

59

In [71]:
# Create new dataframe with only these countries
complete_data_country_list = complete_data_countries.index.to_list()

df_co2_complete = df_co2_time2[df_co2_time2['country'].isin(complete_data_country_list)]

In [73]:
df_co2_complete.head()

,country,year,iso_code,co2,co2_per_capita,co2_growth_abs,coal_co2,oil_co2,gas_co2,cumulative_co2,population
273,Africa,1851,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,112843149.0
274,Africa,1852,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,113379285.0
275,Africa,1853,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,113793047.0
276,Africa,1854,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,114216849.0
277,Africa,1855,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,114631842.0


In [75]:
# Summary of new dataframe
co2_complete_summary = dfSummary(df_co2_complete)
co2_complete_summary

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,country[object],1. Africa2. Oceania (GCP)3. Martinique4. Mauritania5. Mayotte6. Middle East (GCP)7. Netherlands8. Non-OECD (GCP)9. North America10. North America (GCP)11. other,"169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)8,281 (83.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADi0lEQVR4nO3cQWpaYRiG0c8gkkwMIeBiXEQX20W4Gwk6aEomdtC0JJ0UCj4WPWfmIORCHq7cy/9mcTqdBgp3l74AbofYyIiNjNjIiI2M2MiIjYzYyIiNzGJmHmZm9f757XQ6vV7werhiy+fn5y/r9fppZuZ4PL4sFouvguMc7tbr9dN2u33dbrev79Gt/vpT8A+WMzOPj4/f3z8/XPBauHIeEMiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjcxyZuZwONxf+kK4fsvj8fiy2+1+n9SdmbcLXxNXygaBzMK/zKLizkbGuoqMdRUZ6yoyXuqSERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxrFwMgYvZNzZyBi8kDF4IWPwQsZLXTJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4Fk7G4IWMBwQyvkbJWFeRsa4iY11FxgMCGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGesqMtZVZBwLJ2NdRcadjYzBCxmDFzIGL2S81CUjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNj8ELG4IWMY+FkDF7IuLORMXghY/BCxuCFjJe6ZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGxrqKjHUVGRsEMtZVZNzZyFhXkbGuImNdRcZLXTJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4Fk7G4IWMOxsZgxcyBi9kDF7IeKlLRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGxuCFjMELGcfCyRi8kPGAQMbXKBnrKjLWVWSsq8h4QCAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNjXUXGuoqMY+FkrKvIuLORMXghY/BCxuCFjJe6ZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHEsnIzBC5k/72zcpuQb7dPghdtUDZ2WvwYvH3YI3JDD4XC/2+2e9vv9ambOG9vMz8HLZrP5ds5fxH8tGTp5qUtGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbG4OXGlX/7j4MX/0/3RlVDJ8fCmYmOhRu8kPGAQEZsZMRGRmxkxEZGbGTERkZsZMRG5gfEBtMdoo/g2QAAAABJRU5ErkJggg=="">",0(0.0%)
2,year[int64],Mean (sd) : 1935.0 (48.8)min < med < max:1851.0 < 1935.0 < 2019.0IQR (CV) : 84.0 (39.7),169 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACNUlEQVR4nO3Yu47aUBRG4X24yVxkC6FARzNlpBQueYg8bB4hBU9AkSYVHRKyjPCMhTFOkyoage0zM5lfXl/N9rG2lmwLV1WVAZ9d73/fAFDH4NEPnHNjMxu1vH7PzG4tZ33nfc++VFX10mawwzvzmb+777uhOufGi8XiexiG86anlmU5zLLsaTab/e71esVHzvuebWZ2Op0S59yPprF2dWe+84/2/eiJOgrDcL7ZbF6iKMqbHLzf7+e73e5LHMe/VqtV0mTWd9737DRNg+12Oz8ejyMza/pU7eTOfObr7Pvhq9/MLIqifLlcPjc5PEmSsZnZdDptPOs773v2X+OWc2bWvZ29wc7v7rtWqF1UluXQzELnXNPR8Ha7Dd/hljqNUF+RZdmwKIpv6/W63+/3G72+i6IIyrL8muf5TzNr+zTHPwj1FZfLZRAEwSSO47zlt97ker323+v+uohQ7/D51sPb4g9/SCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSBjU+VGapkHTC5/P58DMLMuy4HA4TD5yXvVs1fv2na/T1x807FXfve2AUwAAAABJRU5ErkJggg=="">",0(0.0%)
3,iso_code[object],1. nan2. AND3. GBR4. TUV5. SWE6. ESP7. SGP8. SLE9. REU10. POL11. other,"4,732 (47.5%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)169 (1.7%)3,718 (37.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADyElEQVR4nO3cPW4TUQBG0WdkRdDYsiJ5MV4Ei2URsxsr8hQJojEFP0ooEAJ8J3bO6ege6DJhRu9jdT6fBxTeLX0A3g6xkREbGbGRERsZsZERGxmxkREbmdUY48MY427pg1yBL+fz+WnpQ1yz9f39/cfNZrNb+iCv3TzPD6vV6pPg/t56s9nsDofD03a7/bz0YV6r0+n0fpqm3fF4vBtjiO0vrccYY7vdft7v949LH+aV+7D0Aa6dFwQyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiOzHuPbfa2lD/Ka+fP5P9bzPD9M07Qb7mv91jzPD2OML0uf45rZIPw5G4R/tPJfZlH59cnmby8X82JdZUHEJb37sa46HA5P36Pz7zcu4ue66vuvvZFyMT7qkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTF4IfN88GLUwUW5Fk7G4IWMJxsZgxcyBi9kDF7I+KhLRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGxuCFjMELGdfCyRi8kPGCQMaPUTLWVWSsq8hYV5HxgkBGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbGuoqMdRUZ18LJWFeR8WQjY/BCxuCFjME

In [77]:
# Check for Missing Values
df_co2_complete.isnull().sum().sort_values(ascending=True)

country              0
year                 0
co2                  0
cumulative_co2    1859
co2_growth_abs    1860
co2_per_capita    2309
population        3154
coal_co2          3713
oil_co2           3717
gas_co2           3744
iso_code          4732
dtype: int64

In [79]:
df_co2_complete.shape

(9971, 11)

In [83]:
df_co2_complete.to_csv('/Users/samabrams/Data Analysis Projects/CO2_Emissions_Project/02 Data/Prepared Data/co2_time_series_complete.csv')